

#FidlTrack graph distance pre-computing.

Computes the graph distance between every pair of structure pixel (centres) closer than a specified Euclidean linking distance (cuts on computation time).

Works for single mask image or a stack of mask images. Reduces the size of stacks by applying time-window compression.

The different cells in the pipeline are link so they need to be executed in order and re-executed entirely when parameters change.




**Setup**

In [ ]:
#@markdown Launch this cell to initialise the notebook (you only need to run this once)

from os import path
import numpy as np
import skimage as sk
import struct
from math import floor
import progressbar
from skimage.measure import label, regionprops

import queue
from dataclasses import dataclass, field
from typing import Any

from matplotlib import pyplot as plt

class distTime:
    def __init__(self, d, tstart, tend):
        self.d = d
        self.tstart = tstart
        self.tend = tend

    def __hash__(self):
        return hash(self.d) ^ hash(self.tstart) ^ hash(self.tend)

    def __eq__(self, o):
        return self.d == o.d and self.tstart == o.tstart and self.tend == o.tend

    def insert_in_dict(distt, dict_f, dict_r):
        if distt in dict_f:
            return
        N = len(dict_f.keys())
        dict_f[distt] = N
        dict_r[N] = distt

    def to_list(self):
        return [self.d, self.tstart, self.tend]

def neighbors_1D(px1D, IMSIZE):
    px = [px1D // IMSIZE, px1D % IMSIZE]
    nh = [[px[0] + nh[0], px[1] + nh[1]] for nh in [[0, -1], [-1, 0], [+1, 0], [0, +1],
                                                    [-1, -1], [+1, -1], [-1, +1], [+1, +1]]]
    return [e[0] * IMSIZE + e[1] for e in nh if 0 <= e[0] <= (IMSIZE-1) and 0 <= e[1] <= (IMSIZE-1)]

def iscontained(tr1, tr2):
    return (tr1[0] >= tr2[0] and tr1[0] <= tr2[1] and
            tr1[1] <= tr2[1] and tr1[1] >= tr2[0])


def d1D(px1D, ppx1D, IMSIZE, SQRT2_PXSIZE, PXSIZE):
    return SQRT2_PXSIZE if abs(px1D - ppx1D) in {IMSIZE-1, IMSIZE+1} else PXSIZE

def time_ranges_ovlp(tr1, tr2):
    min_ovlp = max(tr1[0], tr2[0])
    max_ovlp = min(tr1[1], tr2[1])
    if max_ovlp >= min_ovlp:
        return [min_ovlp, max_ovlp]
    return None

def split_timerange(tr1, tr2):
    res = []
    min_ovlp = max(tr1[0], tr2[0])
    max_ovlp = min(tr1[1], tr2[1])

    if max_ovlp < min_ovlp:
        return []

    if tr1[0] < min_ovlp:
        res.append([1, tr1[0], min_ovlp-1])
    elif tr2[0] < min_ovlp:
        res.append([2, tr2[0], min_ovlp-1])
    res.append([12, min_ovlp, max_ovlp])
    if tr1[1] > max_ovlp:
        res.append([1, max_ovlp+1, tr1[1]])
    elif tr2[1] > max_ovlp:
        res.append([2, max_ovlp+1, tr2[1]])
    return res

def find_neighbors(px1D, c, ts, te, pxs_grps, pxs3D, pxs1D_to_3D, imsize):
    res = []
    for nh in neighbors_1D(px1D, imsize):
        if nh not in pxs1D_to_3D:
            continue
        for p3D in pxs1D_to_3D[nh]:
            p1D,j = pxs3D[p3D]
            assert(p1D == nh)
            nh_ts,nh_te,nh_c = pxs_grps[p1D][j]
            t_ovlp = time_ranges_ovlp([ts, te], [nh_ts, nh_te])
            if nh_c == c and t_ovlp != None:
                res.append((p3D, c, t_ovlp[0], t_ovlp[1]))
    return res

def appearance_sets(idxs):
    res = []
    start = idxs[0]
    for i in range(1, len(idxs)):
        if idxs[i-1] + 1 != idxs[i]:
            res.append([start, idxs[i]])
            start = i
    res.append([start, idxs[-1]])
    return res

def tr_len(end, start):
    return end - start

def d2D(px1D, ppx1D, IMSIZE, PXSIZE):
    px2D = [(px1D // IMSIZE) * PXSIZE,  (px1D % IMSIZE) * PXSIZE]
    ppx2D = [(ppx1D // IMSIZE) * PXSIZE,  (ppx1D % IMSIZE) * PXSIZE]
    return np.sqrt((px2D[0] - ppx2D[0])**2 + (px2D[1] - ppx2D[1])**2)

def px3D_1D(frame, px1D, IMSIZE):
    return IMSIZE*IMSIZE * frame + px1D

def merge_ranges(new_elts):
    merge_done = False
    while not merge_done:
        merge_done = True
        to_del = []
        for u in range(len(new_elts)):
            tr1 = new_elts[u][1:3]
            d1 = new_elts[u][0]
            for v in range(len(new_elts)):
                if u == v:
                    continue
                tr2 = new_elts[v][1:3]
                d2 = new_elts[v][0]

                if iscontained(tr1, tr2) and d2 <= d1:
                    to_del = [u]
                    merge_done = False
                    continue
                if iscontained(tr2, tr1) and d1 <= d2:
                    to_del = [v]
                    merge_done = False
                    continue

                isets = split_timerange(tr1, tr2)
                if isets == []:
                    continue
                to_del = [max([u,v]), min([u,v])]
                merge_done = False
                for iset in isets:
                    if iset[0] == 1:
                        new_elts.append([d1, *iset[1:3]])
                    elif iset[0] == 2:
                        new_elts.append([d2, *iset[1:3]])
                    elif iset[0] == 12:
                        new_elts.append([min([d1, d2]), *iset[1:3]])
                    else:
                        assert(False)
            if not merge_done:
                break
        for v in to_del:
            del new_elts[v]

    return new_elts

def reachable_3D(px3D_idx, pxs3D, pxs1D_to_3D, pxs_grps, max_dist, elts_f, elts_r, imsize, pxsize):
    s2pxsize = np.sqrt(2) * pxsize

    res = {}

    start_px1D,j = pxs3D[px3D_idx]
    ts, te, comp = pxs_grps[start_px1D][j]

    done = set()
    done.add(px3D_idx)

    todo = []
    todo.append([(px3D_idx, comp, ts, te), 0])

    while todo != []:
        cur_px3D_idx, comp, ts, te = todo[0][0]
        cur_d = todo[0][1]
        todo = todo[1:]

        px1D,j = pxs3D[cur_px3D_idx]

        for nh_elt in find_neighbors(px1D, comp, ts, te, pxs_grps, pxs3D, pxs1D_to_3D, imsize):
            nh_px3D_idx, nh_comp, nh_ts, nh_te = nh_elt
            nh_px1 = pxs3D[nh_px3D_idx][0]
            nh_d = cur_d + d1D(px1D, nh_px1, imsize, s2pxsize, pxsize)
            distt = distTime(nh_d, nh_ts, nh_te)

            if nh_px1 > start_px1D and nh_d < max_dist:
                if nh_px3D_idx not in res:
                    #firs time we see this pixel
                    distTime.insert_in_dict(distt, elts_f, elts_r)
                    res[nh_px3D_idx] = [elts_f[distt]]
                else:
                    #we have already seen this pixel
                    new_elts = merge_ranges([elts_r[e].to_list() for e in res[nh_px3D_idx]] + [[nh_d, nh_ts, nh_te]])
                    res[nh_px3D_idx] = []
                    for e in new_elts:
                        distt = distTime(e[0], e[1], e[2])
                        distTime.insert_in_dict(distt, elts_f, elts_r)
                        res[nh_px3D_idx].append(elts_f[distt])

            if nh_px3D_idx not in done and nh_d < max_dist:
                todo.append([nh_elt, nh_d])
                done.add(nh_elt[0])

    return res

def find_tw(frame, time_wins, dw):
    for i, c in enumerate(time_wins):
        if frame >= c - dw and frame <= c + dw:
            return i

def pxs2D_to_1D(p, h):
    return p[:,0] * h + p[:,1]

def px1D_to_2D(p1D, h):
    return [p1D // h, p1D % h]

def map_pxs1D_to_3D(px1D, win_idx, pxs_grps, pxs3D, pxs1D_to_3D):
    for i in pxs1D_to_3D[px1D]:
        j = pxs3D[i][1]
        if win_idx >= pxs_grps[px1D][j][0] and win_idx <= pxs_grps[px1D][j][1]:
            return i

def EC_dist_sq(pos1, pos2):
    return np.sum((pos1 - pos2)**2)

def save_bin_comps(w_dur, w_ovlp, nframes, pxs3D, pxs_grps, all_dists, dists, elts_r, rev_map_dists, out_fname):
    with open(out_fname, 'wb') as f:
        f.write(w_dur.to_bytes(3, byteorder='big'))
        f.write(struct.pack('>f', w_ovlp))
        f.write(int(nframes).to_bytes(3, byteorder='big'))

        f.write(len(pxs3D).to_bytes(3, byteorder='big'))
        for px1D, grp_idx in pxs3D:
            f.write(int(px1D).to_bytes(3, byteorder='big'))
            f.write(pxs_grps[px1D][grp_idx][0].to_bytes(3, byteorder='big'))
            f.write(pxs_grps[px1D][grp_idx][1].to_bytes(3, byteorder='big'))

        f.write(len(all_dists).to_bytes(3, byteorder='big'))
        for d in all_dists:
            f.write(struct.pack('>f', d))

        f.write(len(dists).to_bytes(3, byteorder='big'))
        for lab in dists.keys():
            f.write(lab.to_bytes(3, byteorder='big'))
            f.write(len(dists[lab]).to_bytes(3, byteorder='big'))
            for px3D1 in sorted(dists[lab].keys()):
                f.write(int(px3D1).to_bytes(3, byteorder='big'))
                f.write(len(dists[lab][px3D1]).to_bytes(3, byteorder='big'))
                for px3D2 in sorted(dists[lab][px3D1].keys()):
                    f.write(int(px3D2).to_bytes(3, byteorder='big'))
                    f.write(len(dists[lab][px3D1][px3D2]).to_bytes(3, byteorder='big'))
                    for elt in dists[lab][px3D1][px3D2]:
                        distt = elts_r[elt]
                        f.write(rev_map_dists[distt.d].to_bytes(3, byteorder='big'))
                        f.write(distt.tstart.to_bytes(3, byteorder='big'))
                        f.write(distt.tend.to_bytes(3, byteorder='big'))


**Link Google Drive**

In [ ]:
#@markdown Give access to your Google Drive (you only need to run this once)

from google.colab import drive
drive.mount('/content/gdrive')

**Parameters**

When using a single mask image, w_dur must be >= to the number of frames in your SPT recording and w_ovlp = 0.

In [ ]:
mask_img = ""#@param {type:"string"}
max_dist = None#@param {type:"number"}
w_dur = None#@param {type:"number"}
w_ovlp = 0.000#@param {type:"number"}
stab_Nframes = None#@param {type:"number"}
rm_comps_ltpxs = None#@param {type:"number"}
#@markdown Pixel size in µm
pxsize = 0.0967821#@param {type:"number"}
force_recompute = False#@param {type:"boolean"}

if w_ovlp >= 1:
  print("ERROR 0 <= w_ovlp < 1")
  assert(False)

dw = floor(w_dur * (1 - w_ovlp))
if w_ovlp == 0:
    dw = w_dur
print("Window length = {} frame(s)".format(dw))
print("Overlap = {} frame(s)".format(w_dur - dw))

img_dir = path.dirname(mask_img)
img_name = path.basename(mask_img)

stab_fname = path.join(img_dir, "{}_stabN={}".format(path.splitext(img_name)[0], stab_Nframes))
comps_fname = "{}_comps".format(stab_fname)
win_fname = "{}_wDur={}_wOvlp={}".format(comps_fname, w_dur, w_ovlp)
dist_fname = "{}_dist={}".format(win_fname, max_dist)

if not path.isfile(mask_img):
  print("ERROR mask_img file not found: {}".format(mask_img))
  assert(False)

**Stabilisation**

In [ ]:
#@markdown Perform mask stabilisation
if not force_recompute and path.isfile(stab_fname + ".tif"):
  img = sk.io.imread(stab_fname + ".tif")
else:
  img = sk.io.imread(mask_img)
  if img.ndim == 2:
    img = img.reshape((1, img.shape[0], img.shape[1]))

  if stab_Nframes > 1:
    res = np.zeros(img.shape, img.dtype)
    for i in range(img.shape[0]):
      n_left = int(stab_Nframes/2)
      if i < stab_Nframes/2:
          n_left = i
      n_right = int(stab_Nframes/2) + 1
      if i >= img.shape[0] - stab_Nframes/2:
          n_right = img.shape[0] - i

      cur_imgs = img[(i-n_left):(i+n_right), :, :]
      res[i,:,:] = np.max(cur_imgs, axis=0)
    img = res

  sk.io.imsave(stab_fname + ".tif", img, check_contrast=False)
  print("Saved: {}".format(stab_fname + ".tif"))

print("Raw stack length = {}".format(img.shape[0]))

**Compute connected components in time windows**

In [ ]:
#@markdown Compute connected components over time-windows
if not force_recompute and path.isfile(win_fname + ".tif"):
  img = sk.io.imread(win_fname + ".tif")
else:
  win_start = [0] + list(range(dw, img.shape[0], dw))

  labs = np.zeros((len(win_start), img.shape[1], img.shape[2]),
                  dtype="uint16")
  for k, wc in enumerate(win_start):
    for i in range(wc, min(wc + w_dur, img.shape[0])):
      labs[k] = np.maximum(labs[k], img[i] > 0)

    labs[k] = label(labs[k] > 0)
    for p in regionprops(labs[k]):
      if p.coords.shape[0] < rm_comps_ltpxs:
        labs[k, p.coords[:,0], p.coords[:,1]] = 0

  sk.io.imsave(win_fname + ".tif", labs, check_contrast=False)
  print("Saved: {}".format(win_fname + ".tif"))
  with open(win_fname + "_winstart.csv", 'w') as f:
    f.write(",".join([str(e) for e in win_start]) + "\n")
  print("Saved: {}".format(win_fname + "_winstart.csv"))

print("2D mask? {}".format("yes" if labs.shape[0] == 1 else "no"))
print("Number of time windows: {}".format(labs.shape[0]))

**Compute graph distances**

In [ ]:
#@markdown Compute graph distances (this may take a while)

@dataclass(order=True)
class Item:
    priority: int
    v: Any=field(compare=False)

def d1D(px1D, ppx1D, IMSIZE, SQRT2_PXSIZE, PXSIZE):
    return SQRT2_PXSIZE if abs(px1D - ppx1D) in {IMSIZE-1, IMSIZE+1} else PXSIZE

def neighbors_1D(px1D, IMSIZE):
  px = [px1D // IMSIZE, px1D % IMSIZE]
  nh = [[px[0] + nh[0], px[1] + nh[1]] for nh in [[0, -1], [-1, 0], [+1, 0], [0, +1],
                                                  [-1, -1], [+1, -1], [-1, +1], [+1, +1]]]
  return [e[0] * IMSIZE + e[1] for e in nh if 0 <= e[0] <= (IMSIZE-1) and 0 <= e[1] <= (IMSIZE-1)]

def time_ranges_ovlp(tr1, tr2):
  min_ovlp = max(tr1[0], tr2[0])
  max_ovlp = min(tr1[1], tr2[1])
  if max_ovlp >= min_ovlp:
    return [min_ovlp, max_ovlp]
  return None

def find_neighbors3D(px1D, comp, ts, te, pxs_grps, pxs3D, pxs1D_to_3D, imsize):
  res = []
  for nh_px1D in neighbors_1D(px1D, imsize):
    if nh_px1D not in pxs1D_to_3D:
      continue
    for p3D in pxs1D_to_3D[nh_px1D]:
      p1D,j = pxs3D[p3D]
      assert(p1D == nh_px1D)
      nh_ts,nh_te,nh_c = pxs_grps[p1D][j]
      t_ovlp = time_ranges_ovlp([ts, te], [nh_ts, nh_te])
      if nh_c == comp and t_ovlp != None:
        res.append((p3D, p1D, comp, t_ovlp[0], t_ovlp[1]))
  return res

def iscontained(tr1, tr2):
  return (tr1[0] >= tr2[0] and tr1[0] <= tr2[1] and
          tr1[1] <= tr2[1] and tr1[1] >= tr2[0])

def split_timerange(tr1, tr2):
  res = []
  min_ovlp = max(tr1[0], tr2[0])
  max_ovlp = min(tr1[1], tr2[1])

  if max_ovlp < min_ovlp:
    return []

  if tr1[0] < min_ovlp:
    res.append([1, tr1[0], min_ovlp-1])
  elif tr2[0] < min_ovlp:
    res.append([2, tr2[0], min_ovlp-1])
  res.append([12, min_ovlp, max_ovlp])
  if tr1[1] > max_ovlp:
    res.append([1, max_ovlp+1, tr1[1]])
  elif tr2[1] > max_ovlp:
    res.append([2, max_ovlp+1, tr2[1]])
  return res

def merge_ranges(new_elts):
  if len(new_elts) == 1:
    return new_elts

  merge_done = False
  while not merge_done:
    merge_done = True
    to_del = []
    for u in range(len(new_elts)):
      tr1 = new_elts[u][1:3]
      d1 = new_elts[u][0]
      for v in range(len(new_elts)):
        if u == v:
          continue
        tr2 = new_elts[v][1:3]
        d2 = new_elts[v][0]

        if iscontained(tr1, tr2) and d2 <= d1:
          to_del = [u]
          merge_done = False
          continue
        if iscontained(tr2, tr1) and d1 <= d2:
          to_del = [v]
          merge_done = False
          continue

        isets = split_timerange(tr1, tr2)
        if isets == []:
          continue
        to_del = [max([u,v]), min([u,v])]
        merge_done = False
        for iset in isets:
          if iset[0] == 1:
            new_elts.append([d1, *iset[1:3]])
          elif iset[0] == 2:
            new_elts.append([d2, *iset[1:3]])
          elif iset[0] == 12:
            new_elts.append([min([d1, d2]), *iset[1:3]])
          else:
            assert(False)
      if not merge_done:
        break
    for v in to_del:
      del new_elts[v]

  return new_elts

def dijkstra_2D(px1D, pxs1D, max_dist, imsize, pxsize):
  s2pxsize = np.sqrt(2) * pxsize
  idxs = {pxs1D[i]: i for i in range(len(pxs1D))}
  nh1D = np.array([-imsize, -1, +1, imsize, imsize-1, imsize+1,
                   -imsize-1, -imsize+1])

  dists = np.ones((len(pxs1D))) * (2 * max_dist)
  dists[idxs[px1D]] = 0
  q = queue.PriorityQueue()
  q.put(Item(dists[idxs[px1D]], px1D))
  done = np.zeros(len(pxs1D), dtype="bool")

  spxs1D = set(pxs1D)

  while not q.empty():
    cur_px1D = q.get().v

    if done[idxs[cur_px1D]]:
      continue

    done[idxs[cur_px1D]] = 1
    cur_px2D = [cur_px1D // imsize, cur_px1D % imsize]

    for nh_px1D in [e for e in cur_px1D+nh1D if e in spxs1D and done[idxs[e]] == 0]:
      #check if we loop through the image
      nh_px2D = [nh_px1D // imsize, nh_px1D % imsize]
      if abs(cur_px2D[0] - nh_px2D[0]) > 1 or abs(cur_px2D[1] - nh_px2D[1]) > 1:
        continue

      nh_d = dists[idxs[cur_px1D]] + d1D(cur_px1D, nh_px1D, imsize, s2pxsize, pxsize)

      if nh_d < max_dist and nh_d < dists[idxs[nh_px1D]]:
        dists[idxs[nh_px1D]] = nh_d
        q.put(Item(nh_d, nh_px1D))
  return dists

def save_bin_comps2D(all_dists, cache_r, dists, rev_map_dists, out_fname):
  with open(out_fname, 'wb') as f:
    f.write(int(2).to_bytes(3, byteorder='big'))

    f.write(len(all_dists).to_bytes(3, byteorder='big'))
    for d in all_dists:
      f.write(struct.pack('>f', d))

    f.write(len(dists).to_bytes(3, byteorder='big'))
    for lab in sorted(dists.keys()):
      f.write(lab.to_bytes(3, byteorder='big'))
      f.write(len(dists[lab]).to_bytes(3, byteorder='big'))
      for px3D1 in sorted(dists[lab].keys()):
        f.write(int(px3D1).to_bytes(3, byteorder='big'))
        f.write(len(dists[lab][px3D1]).to_bytes(3, byteorder='big'))
        for px3D2 in sorted(dists[lab][px3D1].keys()):
          f.write(int(px3D2).to_bytes(3, byteorder='big'))
          f.write(rev_map_dists[cache_r[dists[lab][px3D1][px3D2]]].to_bytes(3, byteorder='big'))

def save_bin_comps3D(w_dur, w_ovlp, nframes, all_dists, cache_r, dists, rev_map_dists, out_fname):
  with open(out_fname, 'wb') as f:
    f.write(int(1).to_bytes(3, byteorder='big'))
    f.write(w_dur.to_bytes(3, byteorder='big'))
    f.write(struct.pack('>f', w_ovlp))
    f.write(int(nframes).to_bytes(3, byteorder='big'))

    f.write(len(all_dists).to_bytes(3, byteorder='big'))
    for d in all_dists:
      f.write(struct.pack('>f', d))

    f.write(len(dists).to_bytes(3, byteorder='big'))
    for lab in sorted(dists.keys()):
      f.write(lab.to_bytes(3, byteorder='big'))
      f.write(len(dists[lab]).to_bytes(3, byteorder='big'))
      for px1 in sorted(dists[lab].keys()):
        f.write(int(px1).to_bytes(3, byteorder='big'))
        f.write(len(dists[lab][px1]).to_bytes(3, byteorder='big'))
        for px2 in sorted(dists[lab][px1].keys()):
          f.write(int(px2).to_bytes(3, byteorder='big'))
          f.write(len(dists[lab][px1][px2]).to_bytes(3, byteorder='big'))
          for elts in dists[lab][px1][px2]:
            f.write(rev_map_dists[cache_r[elts[0]]].to_bytes(3, byteorder='big'))
            f.write(int(elts[1]).to_bytes(3, byteorder='big'))
            f.write(int(elts[2]).to_bytes(3, byteorder='big'))

####STARTS HERE
SQRT2_PXSIZE = np.sqrt(2) * pxsize
IMSIZE = labs.shape[2]

elts_f = {}
elts_r = {}
dists = {}
if labs.shape[0] == 1: #2D mask
  all_px1Ds = set()
  comps_px1D = {}
  pr = regionprops(labs[0])
  for p in pr:
    px1Ds = set(p.coords[:, 0] * IMSIZE + p.coords[:, 1])
    all_px1Ds.update(px1Ds)
    comps_px1D[p.label] = set(px1Ds)
  all_px1Ds = sorted(all_px1Ds)

  for k in comps_px1D.keys():
    comps_px1D[k] = sorted(comps_px1D[k])

  for comp,pxs in comps_px1D.items():
    comp = int(comp)
    dists[comp] = {}
    for k in progressbar.progressbar(range(len(pxs))):
      tmp = dijkstra_2D(pxs[k], pxs, max_dist, IMSIZE, pxsize)
      tmp = {pxs[i]: np.round(tmp[i],6) for i in range(len(tmp)) if tmp[i] < max_dist and pxs[i] > pxs[k]}

      for d in tmp.values():
        if d not in elts_f:
          ncache = len(elts_f)+1
          elts_f[d] = ncache
          elts_r[ncache] = d
      dists[comp][pxs[k]] = {p: elts_f[d] for p,d in tmp.items()}

  all_dists = set()
  for comp, elts1 in dists.items():
    for px3D1, elts2 in elts1.items():
      all_dists.update([elts_r[didx] for didx in elts2.values()])

  all_dists = sorted(all_dists)
  rev_map_dists = {}
  for i,d in enumerate(all_dists):
    rev_map_dists[d] = i

  save_bin_comps2D(all_dists, elts_r, dists, rev_map_dists, dist_fname + ".bin")
  print("Saved: {}".format(dist_fname + ".bin"))
else: #stack of masks
  for n in progressbar.progressbar(range(labs.shape[0])):
    all_px1Ds = set()
    comps_px1D = {}
    pr = regionprops(labs[n])
    for p in pr:
      px1Ds = set(p.coords[:, 0] * IMSIZE + p.coords[:, 1])
      all_px1Ds.update(px1Ds)
      comps_px1D[p.label] = set(px1Ds)
    all_px1Ds = sorted(all_px1Ds)

    for k in comps_px1D.keys():
      comps_px1D[k] = sorted(comps_px1D[k])

    for comp,pxs in comps_px1D.items():
      comp = int(comp)
      if comp not in dists:
        dists[comp] = {}

      for k in range(len(pxs)):
        tmp = dijkstra_2D(pxs[k], pxs, max_dist, IMSIZE, pxsize)
        tmp = {pxs[i]: np.round(tmp[i],6) for i in range(len(tmp)) if tmp[i] < max_dist and pxs[i] > pxs[k]}

        if tmp != {} and pxs[k] not in dists[comp]:
          dists[comp][pxs[k]] = {}

        for p2,d in tmp.items():
          if d not in elts_f:
            ncache = len(elts_f)+1
            elts_f[d] = ncache
            elts_r[ncache] = d

          if p2 not in dists[comp][pxs[k]]:
            dists[comp][pxs[k]][p2] = {}
          if elts_f[d] not in dists[comp][pxs[k]][p2]:
            dists[comp][pxs[k]][p2][elts_f[d]] = []
          dists[comp][pxs[k]][p2][elts_f[d]].append(n)

      if len(dists[comp]) == 0:
        del dists[comp]

  dists_tmp = dists

  for c in dists.keys():
    for p1 in dists[c].keys():
      for p2 in dists[c][p1].keys():
        tmp = []
        for d,elts in dists[c][p1][p2].items():
          start = elts[0]
          elts = sorted(elts)
          for i in range(1, len(elts)):
            if elts[i-1] != elts[i] - 1:
              tmp.append((d, start, elts[i-1]))
              start = elts[i]
          tmp.append((d, start, elts[len(elts)-1]))
        dists[c][p1][p2] = tmp

  all_dists = set()
  for comp, elts1 in dists.items():
    for px3D1, elts2 in elts1.items():
      for px3D2, elts3 in elts2.items():
        [all_dists.add(elts_r[elt[0]]) for elt in elts3]

  all_dists = sorted(all_dists)
  rev_map_dists = {}
  for i,d in enumerate(all_dists):
    rev_map_dists[d] = i

  save_bin_comps3D(w_dur, w_ovlp, labs.shape[0], all_dists, elts_r, dists, rev_map_dists, dist_fname + ".bin")
  print("Saved: {}".format(dist_fname + ".bin"))